<a href="https://colab.research.google.com/github/DylanZol/CSC423-Final-Project/blob/main/ProjectPart3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import sqlite3

# Connect to (or create) the SQLite database file
conn = sqlite3.connect("supermaids.db")

conn.execute("PRAGMA foreign_keys = ON;")
cur = conn.cursor()

print("Connected to SQLite, foreign keys ON.")

Connected to SQLite, foreign keys ON.


In [53]:
# schema (DROP + CREATE TABLE statements)

schema_sql = """
DROP TABLE IF EXISTS Assigned_To;
DROP TABLE IF EXISTS Requires_Equipment;
DROP TABLE IF EXISTS Requirements;
DROP TABLE IF EXISTS Equipment;
DROP TABLE IF EXISTS Employees;
DROP TABLE IF EXISTS Clients;

CREATE TABLE Clients (
    clientNo      TEXT PRIMARY KEY,
    firstName     TEXT NOT NULL,
    lastName      TEXT NOT NULL,
    address       TEXT NOT NULL,
    phoneNumber   TEXT NOT NULL,
    preferences   TEXT,
    feedback      TEXT
);

CREATE TABLE Employees (
    staffNo      TEXT PRIMARY KEY,
    firstName    TEXT NOT NULL,
    lastName     TEXT NOT NULL,
    address      TEXT NOT NULL,
    salary       REAL NOT NULL CHECK (salary > 0),
    phoneNumber  TEXT NOT NULL
);

CREATE TABLE Equipment (
    equipmentID  TEXT PRIMARY KEY,
    description  TEXT NOT NULL,
    usage        TEXT,
    cost         REAL NOT NULL CHECK (cost >= 0),
    status       TEXT NOT NULL CHECK (
                     status IN ('available','in_use','maintenance','out_of_service')
                 )
);

CREATE TABLE Requirements (
    requirementID  TEXT PRIMARY KEY,
    clientNo       TEXT NOT NULL,
    startDate      TEXT NOT NULL,  -- 'YYYY-MM-DD'
    startTime      TEXT NOT NULL,  -- 'HH:MM'
    duration       INTEGER NOT NULL CHECK (duration > 0), -- minutes or hours
    comments       TEXT,
    priority       TEXT NOT NULL CHECK (
                       priority IN ('high','medium','low')
                   ),
    FOREIGN KEY (clientNo)
        REFERENCES Clients(clientNo)
        ON DELETE RESTRICT
        ON UPDATE CASCADE
);

CREATE TABLE Requires_Equipment (
    requirementID    TEXT NOT NULL,
    equipmentID      TEXT NOT NULL,
    quantityNeeded   INTEGER NOT NULL CHECK (quantityNeeded > 0),
    occasionsNeeded  INTEGER NOT NULL CHECK (occasionsNeeded > 0),
    totalOccasions   INTEGER NOT NULL CHECK (
                          totalOccasions > 0
                          AND occasionsNeeded <= totalOccasions
                      ),
    PRIMARY KEY (requirementID, equipmentID),
    FOREIGN KEY (requirementID)
        REFERENCES Requirements(requirementID)
        ON DELETE RESTRICT
        ON UPDATE CASCADE,
    FOREIGN KEY (equipmentID)
        REFERENCES Equipment(equipmentID)
        ON DELETE RESTRICT
        ON UPDATE CASCADE
);

CREATE TABLE Assigned_To (
    staffNo        TEXT NOT NULL,
    requirementID  TEXT NOT NULL,
    role           TEXT NOT NULL CHECK (
                       role IN ('team_leader','cleaner','inspector','trainee')
                   ),
    PRIMARY KEY (staffNo, requirementID),
    FOREIGN KEY (staffNo)
        REFERENCES Employees(staffNo)
        ON DELETE RESTRICT
        ON UPDATE CASCADE,
    FOREIGN KEY (requirementID)
        REFERENCES Requirements(requirementID)
        ON DELETE RESTRICT
        ON UPDATE CASCADE
);
"""

cur.executescript(schema_sql)
conn.commit()
print("Schema created / reset.")


Schema created / reset.


In [54]:
#  sample data (5 tuples per relation)

insert_sql = """
-- CLIENTS
INSERT INTO Clients VALUES
('C001','Alice','Campos','123 Main St','305-111-1111','eco-friendly products','Great service so far'),
('C002','Brian','Lopez','45 Sunset Ave','305-222-2222','evening only',NULL),
('C003','Carla','Nguyen','78 Ocean Dr','305-333-3333',NULL,'Unhappy with last cleaner'),
('C004','David','Singh','90 Pine Rd','305-444-4444','pet safe','Loves team leader Emma'),
('C005','Ella','Martin','12 Elm St','305-555-5555','low noise equipment',NULL);

-- EMPLOYEES
INSERT INTO Employees VALUES
('S001','Emma','Reyes','1 Oak St', 22.50,'786-111-1111'),
('S002','Jared','Kim','2 Oak St', 18.00,'786-222-2222'),
('S003','Natalie','Stone','3 Oak St',19.75,'786-333-3333'),
('S004','Omar','Diaz','4 Oak St', 21.00,'786-444-4444'),
('S005','Priya','Patel','5 Oak St',17.25,'786-555-5555');

-- EQUIPMENT
INSERT INTO Equipment VALUES
('E001','Industrial floor cleaner','floors',1500.00,'available'),
('E002','Steam cleaner','bathrooms',900.00,'available'),
('E003','Vacuum','carpets',400.00,'in_use'),
('E004','Mop set','general',120.00,'available'),
('E005','Eco-friendly sprayer','surfaces',250.00,'maintenance');

-- REQUIREMENTS
INSERT INTO Requirements VALUES
('R001','C001','2025-01-05','09:00',180,'Weekly full clean','high'),
('R002','C002','2025-01-05','18:00',120,'Evening office clean','medium'),
('R003','C003','2025-01-06','10:00',90,'Kitchen and bathrooms','high'),
('R004','C004','2025-01-06','13:00',120,'Pet-friendly deep clean','high'),
('R005','C005','2025-01-07','08:30',60,'Quick morning tidy','low');

-- REQUIRES_EQUIPMENT
INSERT INTO Requires_Equipment VALUES
('R001','E001',2,4,4),
('R001','E004',4,4,4),
('R002','E003',3,2,2),
('R003','E002',1,1,1),
('R004','E005',1,1,1);

-- ASSIGNED_TO
INSERT INTO Assigned_To VALUES
('S001','R001','team_leader'),
('S002','R001','cleaner'),
('S003','R002','team_leader'),
('S004','R003','team_leader'),
('S005','R004','trainee');
"""

cur.executescript(insert_sql)
conn.commit()
print("Sample data inserted.")


Sample data inserted.


In [55]:

# CELL 4 – Helper: pretty-print any table

def show_table(table_name):
    print(f"\n=== {table_name} ===")
    cur.execute(f"PRAGMA table_info({table_name});")
    cols = [row[1] for row in cur.fetchall()]
    print(" | ".join(cols))
    print("-" * (4 * len(cols) + 5))

    cur.execute(f"SELECT * FROM {table_name};")
    for row in cur.fetchall():
        print(" | ".join(str(x) if x is not None else "NULL" for x in row))

# Example: show all tables
for t in ["Clients","Employees","Equipment","Requirements","Requires_Equipment","Assigned_To"]:
    show_table(t)



=== Clients ===
clientNo | firstName | lastName | address | phoneNumber | preferences | feedback
---------------------------------
C001 | Alice | Campos | 123 Main St | 305-111-1111 | eco-friendly products | Great service so far
C002 | Brian | Lopez | 45 Sunset Ave | 305-222-2222 | evening only | NULL
C003 | Carla | Nguyen | 78 Ocean Dr | 305-333-3333 | NULL | Unhappy with last cleaner
C004 | David | Singh | 90 Pine Rd | 305-444-4444 | pet safe | Loves team leader Emma
C005 | Ella | Martin | 12 Elm St | 305-555-5555 | low noise equipment | NULL

=== Employees ===
staffNo | firstName | lastName | address | salary | phoneNumber
-----------------------------
S001 | Emma | Reyes | 1 Oak St | 22.5 | 786-111-1111
S002 | Jared | Kim | 2 Oak St | 18.0 | 786-222-2222
S003 | Natalie | Stone | 3 Oak St | 19.75 | 786-333-3333
S004 | Omar | Diaz | 4 Oak St | 21.0 | 786-444-4444
S005 | Priya | Patel | 5 Oak St | 17.25 | 786-555-5555

=== Equipment ===
equipmentID | description | usage | cost | stat

In [56]:
# QUERY 1 – All requirements for a given client

client_no = "C001"   # change to test others

query1 = """
SELECT  r.requirementID,
        r.startDate,
        r.startTime,
        r.duration,
        r.priority,
        r.comments
FROM Requirements r
WHERE r.clientNo = ?
ORDER BY r.startDate, r.startTime;
"""

print("\nRequirements for client", client_no)
for row in cur.execute(query1, (client_no,)):
    print(row)

# QUERY 2 – Full schedule for a given day (with client name)

date = "2025-01-05"   # change as needed

query2 = """
SELECT  r.requirementID,
        c.clientNo,
        c.firstName || ' ' || c.lastName AS clientName,
        r.startTime,
        r.duration,
        r.priority
FROM Requirements r
JOIN Clients c ON r.clientNo = c.clientNo
WHERE r.startDate = ?
ORDER BY r.startTime;
"""

print("\nSchedule for", date)
for row in cur.execute(query2, (date,)):
    print(row)



# QUERY 3 – Equipment needed for a given requirement

req_id = "R001"

query3 = """
SELECT  e.equipmentID,
        e.description,
        re.quantityNeeded,
        re.occasionsNeeded,
        re.totalOccasions,
        e.status
FROM Requires_Equipment re
JOIN Equipment e ON re.equipmentID = e.equipmentID
WHERE re.requirementID = ?;
"""

print("\nEquipment for requirement", req_id)
for row in cur.execute(query3, (req_id,)):
    print(row)



# QUERY 4 – Staff assigned to a given requirement and their roles


req_id = "R001"

query4 = """
SELECT  a.staffNo,
        emp.firstName || ' ' || emp.lastName AS employeeName,
        a.role
FROM Assigned_To a
JOIN Employees emp ON a.staffNo = emp.staffNo
WHERE a.requirementID = ?;
"""

print("\n Staff assigned to requirement", req_id)
for row in cur.execute(query4, (req_id,)):
    print(row)


# QUERY 5 – Employees who have worked for a given client


client_no = "C003"

query5 = """
SELECT DISTINCT e.staffNo,
       e.firstName || ' ' || e.lastName AS employeeName
FROM Requirements r
JOIN Assigned_To a ON r.requirementID = a.requirementID
JOIN Employees e   ON a.staffNo       = e.staffNo
WHERE r.clientNo = ?;
"""

print("\n Employees who have worked for client", client_no)
for row in cur.execute(query5, (client_no,)):
    print(row)





Requirements for client C001
('R001', '2025-01-05', '09:00', 180, 'high', 'Weekly full clean')

Schedule for 2025-01-05
('R001', 'C001', 'Alice Campos', '09:00', 180, 'high')
('R002', 'C002', 'Brian Lopez', '18:00', 120, 'medium')

Equipment for requirement R001
('E001', 'Industrial floor cleaner', 2, 4, 4, 'available')
('E004', 'Mop set', 4, 4, 4, 'available')

 Staff assigned to requirement R001
('S001', 'Emma Reyes', 'team_leader')
('S002', 'Jared Kim', 'cleaner')

 Employees who have worked for client C003
('S004', 'Omar Diaz')


In [57]:

conn.close()
print("Connection closed.")


Connection closed.
